In [1]:
from edgar.stock import Stock
from edgar.financials import FinancialReportEncoder
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import json
import datetime

In [2]:
stock = Stock('AAPL')

period = 'quarterly' # or 'annual', which is the default
year = 2019 # can use default of 0 to get the latest
quarter = [1, 2, 3]   #or default value of 0 to get the latest

income_list = []

for q in quarter:
    print(q)
    # using defaults to get the latest annual, can simplify to stock.get_filing()
    filing = stock.get_filing(period, year, q)

    # financial reports (contain data for multiple years)
    income_statements = filing.get_income_statements()
    balance_sheets = filing.get_balance_sheets()
    cash_flows = filing.get_cash_flows()
    
    income_statements_json = FinancialReportEncoder().encode(income_statements)
    income_statements_json = json.loads(income_statements_json)
    
    income = income_statements_json['reports']
    
    income_list.append(income)

cik for AAPL is 320193
1
getting ['10-Q', '10-Q/A'] filing info from https://www.sec.gov/Archives/edgar/full-index/2019/QTR1/master.idx
Processing SGML at https://www.sec.gov/Archives/edgar/data/320193/0000320193-19-000010.txt
could not find anything for ShortName consolidated statements of income
could not find anything for ShortName consolidated statements of operations
could not find anything for ShortName consolidated statement of earnings
could not find anything for ShortName condensed consolidated statements of income (unaudited)
could not find anything for ShortName condensed consolidated statements of income
could not find anything for ShortName condensed consolidated statements of operations
could not find anything for ShortName condensed consolidated statement of earnings (unaudited)
could not find anything for ShortName condensed consolidated statement of earnings
could not find anything for ShortName condensed statements of income
could not find anything for ShortName conde

In [3]:
income_list

[[{'date': '2018-12-29T00:00:00',
   'months': 3,
   'map': {'us-gaap_RevenueFromContractWithCustomerExcludingAssessedTax': {'label': 'Net sales',
     'value': 84310000000.0},
    'us-gaap_CostOfGoodsAndServicesSold': {'label': 'Cost of sales',
     'value': 52279000000.0},
    'us-gaap_GrossProfit': {'label': 'Gross margin', 'value': 32031000000.0},
    'us-gaap_ResearchAndDevelopmentExpense': {'label': 'Research and development',
     'value': 3902000000.0},
    'us-gaap_SellingGeneralAndAdministrativeExpense': {'label': 'Selling, general and administrative',
     'value': 4783000000.0},
    'us-gaap_OperatingExpenses': {'label': 'Total operating expenses',
     'value': 8685000000.0},
    'us-gaap_OperatingIncomeLoss': {'label': 'Operating income',
     'value': 23346000000.0},
    'us-gaap_NonoperatingIncomeExpense': {'label': 'Other income/(expense), net',
     'value': 560000000.0},
    'us-gaap_IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingI

In [4]:
# initialize
income_df = pd.DataFrame(columns=['Date','Profit', 'Net Sales', 'Operating Income', 'Net Income'])

for il in income_list:
    il = [element for element in il if element['months']==3]
    for element in il:
        print(element['date'], element['months'])
        profit = element['map']['us-gaap_GrossProfit']['value']
        net_sales = element['map']['us-gaap_RevenueFromContractWithCustomerExcludingAssessedTax']['value']
        operating_income = element['map']['us-gaap_OperatingIncomeLoss']['value']
        net_income = element['map']['us-gaap_NetIncomeLoss']['value']
        print(net_income)
        
        temp_df = pd.DataFrame({
            'Profit' : [profit],
            'Net Sales' : [net_sales],
            'Operating Income' : [operating_income],
            'Net Income' : [net_income],
            'Date' : list(pd.to_datetime([element['date']]))
        })
    
        income_df = income_df.append(temp_df)

income_df

2018-12-29T00:00:00 3
19965000000.0
2017-12-30T00:00:00 3
20065000000.0
2019-03-30T00:00:00 3
11561000000.0
2018-03-31T00:00:00 3
13822000000.0
2019-06-29T00:00:00 3
10044000000.0
2018-06-30T00:00:00 3
11519000000.0


C:\Users\IEshun\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


,Date,Net Income,Net Sales,Operating Income,Profit
0,2018-12-29,1.996500e+10,8.431000e+10,2.334600e+10,3.203100e+10
0,2017-12-30,2.006500e+10,8.829300e+10,2.627400e+10,3.391200e+10
0,2019-03-30,1.156100e+10,5.801500e+10,1.341500e+10,2.182100e+10
0,2018-03-31,1.382200e+10,6.113700e+10,1.589400e+10,2.342200e+10
0,2019-06-29,1.004400e+10,5.380900e+10,1.154400e+10,2.022700e+10
0,2018-06-30,1.151900e+10,5.326500e+10,1.261200e+10,2.042100e+10


In [5]:
h_df = pd.DataFrame({
            'Profit' : [24084000000, 24313000000],
            'Net Sales' : [62900000000, 64040000000],
            'Operating Income' : [70898000000, 63930000000],
            'Net Income' : [14125000000, 13686000000],
            'Date' : list(pd.to_datetime(['2018-09-28', '2019-09-27']))
        })
    
income_df = income_df.append(h_df)
income_df

,Date,Net Income,Net Sales,Operating Income,Profit
0,2018-12-29,1.996500e+10,8.431000e+10,2.334600e+10,3.203100e+10
0,2017-12-30,2.006500e+10,8.829300e+10,2.627400e+10,3.391200e+10
0,2019-03-30,1.156100e+10,5.801500e+10,1.341500e+10,2.182100e+10
0,2018-03-31,1.382200e+10,6.113700e+10,1.589400e+10,2.342200e+10
0,2019-06-29,1.004400e+10,5.380900e+10,1.154400e+10,2.022700e+10
0,2018-06-30,1.151900e+10,5.326500e+10,1.261200e+10,2.042100e+10
0,2018-09-28,1.412500e+10,6.290000e+10,7.089800e+10,2.408400e+10
1,2019-09-27,1.368600e+10,6.404000e+10,6.393000e+10,2.431300e+10


In [6]:
income_df.reset_index(inplace = True, drop = True)

In [7]:
income_df['Year'] = income_df['Date'].dt.year
income_df['Quarter'] = income_df['Date'].dt.quarter
income_df

,Date,Net Income,Net Sales,Operating Income,Profit,Year,Quarter
0,2018-12-29,1.996500e+10,8.431000e+10,2.334600e+10,3.203100e+10,2018,4
1,2017-12-30,2.006500e+10,8.829300e+10,2.627400e+10,3.391200e+10,2017,4
2,2019-03-30,1.156100e+10,5.801500e+10,1.341500e+10,2.182100e+10,2019,1
3,2018-03-31,1.382200e+10,6.113700e+10,1.589400e+10,2.342200e+10,2018,1
4,2019-06-29,1.004400e+10,5.380900e+10,1.154400e+10,2.022700e+10,2019,2
5,2018-06-30,1.151900e+10,5.326500e+10,1.261200e+10,2.042100e+10,2018,2
6,2018-09-28,1.412500e+10,6.290000e+10,7.089800e+10,2.408400e+10,2018,3
7,2019-09-27,1.368600e+10,6.404000e+10,6.393000e+10,2.431300e+10,2019,3


In [9]:
#export to csv
income_df.to_csv("../is.csv",index=False)